In [ ]:
# 【0】Python爬虫架构：
'''
网页解析器：将一个网页字符串进行解析，可以按照我们的要求来提取出我们有用的信息，也可以根据DOM树的解析方式来解析。
            网页解析器有:
            (1)正则表达式（直观，将网页转成字符串通过模糊匹配的方式来提取有价值的信息， 当文档比较复杂的时候，
            该方法提取数据的时候就会非常的困难）、
            (2) html.parser（Python自带的）、
            (3) beautifulsoup（第三方插件）、
            (4) lxml（第三方插件，可以解析 xml 和 HTML）
            html.parser 和 beautifulsoup 以及 lxml 都是以 DOM 树的方式进行解析的。
           （5）Nutch、pyspider、scrap
Python爬虫10步走
基础知识。包括python基本语法，html常识，重点理解python字符串处理常用方法和2种数据结构：dict（字典）和json。
3个IDE：Idle，Sublime，Pycharm, Jupyter notebook , Vscode , notepad++ ,spyder 
2个爬虫库：urllib 和requests。
3种解析方法：beautifufsoup，lxml.etree.xpath，re
            lxml.estree-xpath适用于结构规律性较强的网页；
            re采用正则匹配，只要能找出唯一适用的正则表达式即可；
            beautifulsoup使用也较为简单
2种抓包分析方法：F12和fiddler。各浏览器F12后都会进入开发者工具，既可以查看具体的html源码，
            也能分析浏览器和服务器端交互过程，包括request方法，headers，cookie，post data等等，
            需熟练运用。当面对一些复杂页面或者移动端数据（例如手机APP）时，简单的F12工具则不足以满足需求，
            此时可以借助抓包工具Fiddler，设置好代理后就能就能将移动端所有数据交互过程全部记录下来，便于分析和抓取。
2个数据库：MySQL和MongoDB。爬虫数据量较大时，简单的存储至本地文件不是一个理想的选择，
            这时就可视具体抓取数据格式决定选用其中一种数据库。
爬虫身份证：cookie。主要是应对一些必须要登录后方可爬取数据的平台，如知乎、淘宝等，这时就必须
            携带一个登录后的cookie。cookie是一个字典格式的数据，简单的可直接复制就能使用，
            复杂的可以解析post data后构造，甚至是应用selenium模拟登录后得到。
所见即所得：selenium。为了得到爬虫的身份cookie，一些网站的post data又进行了复杂的加密，它几乎可以完全模拟人工登录
            的过程，从而实现指哪抓哪。若是用可视的浏览器模式，则可以看到浏览器自动化操作的场面；
            若是用headless模式，则是通常意义的爬虫模式。
程式化爬虫框架：scrapy。若要进行全站抓取，应用scrapy爬虫框架可以事半功倍。实际上，scrapy是一个爬虫模板，
            通过定制其中的一些设置后（如爬取初始页，抓取数据格式，抓取方法等）即可以快速实现全站抓取。
            scrapy用得熟练之后，也十分有利于培养爬虫工程师思维。
高效爬虫：threading。就像计算机最终从单核走向了多核一样，爬虫最终还是要走向多线程乃至分布式。尤其是待爬取的任务
            工作量大而又重复度很高时，就非常适用多线程爬虫，相当于从一只虫子变成了多只虫子在爬，效率提升明显。
'''

In [ ]:
# 【0】爬虫伪装技巧
1.浏览器伪装
因为网站服务器能够很轻易的识别出访问的来源浏览器，以requests请求为例，默认header头数据中没有浏览器信息，在与浏览器交互时简直就是“裸奔”，所以我们可以加入“User-Agent”信息伪装成真实浏览器，代码如下：

import requests 
headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:88.0) Gecko/20100101 Firefox/88.0'}  #模拟成火狐浏览器
response = requests.get("http://www.baidu.com",headers=headers)  #模拟请求url
# ————————————————————————————
2.访问地址伪装
访问地址指的是headers头部中的reffer信息，那么它有什么作用呢？举个例子解释一下：
我在https://bj.meituan.com/里有一个https://waimai.meituan.com/链接，那么点击这个https://waimai.meituan.com/，
它的header信息里就有：Referer=https://bj.meituan.com/ 
那么可以利用这个来防止盗链，比如我只允许我自己的网站访问我自己的图片服务器
我们可以加入“reffer”信息伪装访问地址，代码如下：

import requests 
headers={
    'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:88.0) Gecko/20100101 Firefox/88.0',
    'reffer':'https://bj.meituan.com/'}
response = requests.get("https://waimai.meituan.com/",headers=headers)  #模拟请求url
# ————————————————————————————

3.ip地址伪装
对于网络中的反爬虫策略来说，大多数都是根据单个IP的行为来判断是不是网络爬虫的，例如，反爬虫检测到某个IP的
访问次数很多，或者是访问的频率很快，就会封禁这个IP。这时我们就要选择代理IP来突破反爬虫的机制，更稳定的及逆行数据
的爬取。代理IP可以自己去网上找免费的，但不太稳定，也可去花钱买一些比较稳定的。
python添加代理IP的代码如下：

import requests 
proxies={'https':'101.236.54.97:8866'} 
headers={
    'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:88.0) Gecko/20100101 Firefox/88.0',
    'reffer':'https://bj.meituan.com/'}
response = requests.get("https://waimai.meituan.com/",headers=headers，proxies=proxies)  #模拟请求url
# ————————————————————————————
4.伪装访问速率
真实用户的访问次数以及访问规律是很稳定的，并不会多次的访问，所以我们要伪装成真实的用户来爬取数据，这样反爬虫机制
就不会察觉，可以采用控制访问频率的方式，主要是随机设置访问时间，代码如下：

import requests 
import time,random
proxies={'https':'101.236.54.97:8866'} 
headers={
    'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:88.0) Gecko/20100101 Firefox/88.0',
    'reffer':'https://bj.meituan.com/'}
for i in range(10):
    response = requests.get("https://waimai.meituan.com/",headers=headers，proxies=proxies)  #模拟请求url
    time.sleep(random.uniform(1.1,5.4))

# ————————————————————————————
5.伪装用户真实信息
有些网页是需要登录后才会显示数据，而cookie值会携带个人的登录信息，在爬虫中加入cookie值就能避免登录的麻烦，
例如知乎、京东等网站，加入方法如下：

import requests 
proxies={'https':'101.236.54.97:8866'} 
headers={
    'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:88.0) Gecko/20100101 Firefox/88.0',
    'reffer':'https://bj.meituan.com/'}
cookies=''
response = requests.get("https://waimai.meituan.com/",headers=headers，proxies=proxies,,cookies=cookies)  #模拟请求url
# ————————————————————————————


In [ ]:
# 【1】requests + random + bs4 + re + lxml.etree 获取房价："http://www.ke.com"
import random
import requests
from bs4 import BeautifulSoup
import re
import math
from lxml import etree

USER_AGENTS = [
    "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; AcooBrowser; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)",
    "Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AOLBuild 4337.35; Windows NT 5.1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
    "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)",
    "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)",
    "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/535.20 (KHTML, like Gecko) Chrome/19.0.1036.7 Safari/535.20",
    "Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; fr) Presto/2.9.168 Version/11.52",]
chinese_city_district_dict = dict()
chinese_area_dict = dict() 

def create_headers():
    headers = dict()
    headers["User-Agent"] = random.choice(USER_AGENTS)
    headers["Referer"] = "http://www.ke.com"
    return headers
class SecHouse(object):
    def __init__(self, district, area, name, price, desc, pic):
        self.district = district
        self.area = area
        self.price = price
        self.name = name
        self.desc = desc
        self.pic = pic
    def text(self):
        return self.district + "," + \
                self.area + "," + \
                self.name + "," + \
                self.price + "," + \
                self.desc + "," + \
                self.pic
def get_districts():
    url = 'https://bj.ke.com/xiaoqu/'
    headers = create_headers()
    response = requests.get(url, timeout=10, headers=headers)
    html = response.content
    root = etree.HTML(html)
    elements = root.xpath('//div[@data-role="ershoufang"]/div/a')
    en_names = list()
    ch_names = list()
    for element in elements:
        link = element.attrib['href']
        en_names.append(link.split('/')[-2])
        ch_names.append(element.text)

    # 打印区县英文和中文名列表
    for index, name in enumerate(en_names):
        chinese_city_district_dict[name] = ch_names[index]
    return en_names

def get_areas(district):
    page = "http://bj.ke.com/xiaoqu/{0}".format(district)
    areas = list()
    try:
        headers = create_headers()
        response = requests.get(page, timeout=10, headers=headers)
        html = response.content
        root = etree.HTML(html)
        links = root.xpath('//div[@data-role="ershoufang"]/div[2]//a')
        # 针对a标签的list进行处理
        for link in links:
            relative_link = link.attrib['href']
            relative_link = relative_link[:-1]   # 去掉最后的"/"
            area = relative_link.split("/")[-1]  # 获取最后一节 英文县城
            if area != district:                 # 去掉区县名,防止重复
                chinese_area = link.text   # 获取中文县城名
                chinese_area_dict[area] = chinese_area
                # print(chinese_area)
                areas.append(area)
        return areas
    except Exception as e:
        print(e)
'''_______________ 测试______________________'''    
# districts = get_districts()
# print(districts)                   # 城区的英文名列表
# print(chinese_city_district_dict)  # 城区的中英文字典 'dongcheng': '东城'
# get_area=get_areas("dongcheng")    
# print(get_area)              # 县城的英文名列表
# print(chinese_area_dict)     # 县城的中英文字典  'andingmen': '安定门'
'''_________________保存数据____________________'''
with open(r"C:\Users\hp\Desktop\sechouse.txt", "w", encoding='utf-8') as f:
# 开始获得需要的板块数据
    total_page = 1
    sec_house_list = list()
    districts = get_districts()         # 城区的英文名列表
    for district in districts[0:1]:
        arealist = get_areas(district)  # 县城的英文名列表
        for area in arealist[0:1]:
# 中文区县: 城区的中英文字典 'dongcheng': '东城'
            chinese_district = chinese_city_district_dict[district]
#             print(chinese_district)
# 中文版块: 县城的中英文字典  'andingmen': '安定门'
            chinese_area = chinese_area_dict.get(area, "")
#             print(chinese_area)
            page = 'http://bj.ke.com/ershoufang/{0}/'.format(area)
            headers = create_headers()
            response = requests.get(page, timeout=10, headers=headers)
            html = response.content
            soup = BeautifulSoup(html, "lxml")
# # 获得总的页数
            try:
                page_box = soup.find_all('div', class_='page-box')[0]
                matches = re.search('totalPage":(\d+)', str(page_box))
                total_page = int(matches.group(1))
            except Exception as e:
                print(e)
            print(total_page)
# # 从第一页开始,一直遍历到最后一页
            headers = create_headers()
            for i in range(1, total_page + 1):
                page = 'http://bj.ke.com/ershoufang/{0}/pg{1}'.format(area,i)
                print("正在获取第",page,"页")
                response = requests.get(page, timeout=10, headers=headers)
                html = response.content
                soup = BeautifulSoup(html, "lxml")
                # 获得有小区信息的panel
                house_elements = soup.find_all('li', class_="clear")
                for house_elem in house_elements:
                    price = house_elem.find('div', class_="totalPrice")
                    name = house_elem.find('div', class_='title')
                    desc = house_elem.find('div', class_="houseInfo")
                    pic = house_elem.find('a', class_="img").find('img', class_="lj-lazy")
# 继续清理数据
                    price = price.text.strip()
                    name = name.text.replace("\n", "")
                    desc = desc.text.replace("\n", "").strip()
                    pic = pic.get('data-original').strip()
# # 作为对象保存
                    sec_house = SecHouse(chinese_district, chinese_area, name, price, desc, pic)
                    print(sec_house.text())
                    sec_house_list.append(sec_house)   
    for sec_house in sec_house_list:
        f.write(sec_house.text() + "\n")

In [ ]:
# 【2】Python爬取字节跳动1W+招聘信息：x-csrf-token
import requests
from urllib.parse import unquote
import pandas as pd
import time
import os

session = requests.session()
page = 10
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36',
    'Origin': 'https://jobs.bytedance.com',
    'Referer': f'https://jobs.bytedance.com/experienced/position?keywords=&category=&location=&project=&type=&job_hot_flag=&current=1&limit={page}'}
data = {"portal_entrance": 1}
url = "https://jobs.bytedance.com/api/v1/csrf/token"
r = session.post(url, headers=headers, data=data)
cookies = session.cookies.get_dict()

url = "https://jobs.bytedance.com/api/v1/search/job/posts"
headers["x-csrf-token"] = unquote(cookies["atsx-csrf-token"])
data = {
    "job_category_id_list": [],
    "keyword": "",
    "limit": page,
    "location_code_list": [],
    "offset": 0,
    "portal_entrance": 1,
    "portal_type": 2,
    "recruitment_id_list": [],
    "subject_id_list": [] }
for i in range(5):
    print(f"准备爬取第{i}页")
    data["offset"] = i*page
    r = None
    while not r:
        try:
            r = session.post(url, headers=headers,data=json.dumps(data), timeout=3)
        except Exception as e:
            print("访问超时！等待5s", e)
            time.sleep(5)
    df = pd.DataFrame(r.json()['data']['job_post_list'])
    if df.shape[0] == 0:
        print("爬取完毕！！！")
        break
    df.city_info = df.city_info.str['name']
    df.recruit_type = df.recruit_type.str['parent'].str['name']
    tmp = []
    for x in df.job_category.values:
        if x['parent']:
            tmp.append(f"{x['parent']['name']}-{x['name']}")
        else:
            tmp.append(x['name'])
    df.job_category = tmp
    df.publish_time = df.publish_time.apply(lambda x: pd.Timestamp(x, unit="ms"))
    df.drop(columns=['sub_title', 'job_hot_flag', 'job_subject'], inplace=True)
    df.to_csv(r"C:\Users\hp\Desktop\bytedance_jobs.csv", mode="a", encoding='utf_8_sig',
              header=not os.path.exists(r"C:\Users\hp\Desktop\bytedance_jobs.csv"), index=False)
    print(",".join(df.title.head(10)))
# 对结果去重
df = pd.read_csv(r"C:\Users\hp\Desktop\bytedance_jobs.csv",encoding='utf_8_sig',)
df.drop_duplicates(inplace=True)
df.to_csv(r"C:\Users\hp\Desktop\bytedance_jobs.csv", index=False,encoding='utf_8_sig',)
print("共爬取", df.shape[0], "行无重复数据")